In [47]:
import pandas as pd
from surprise.prediction_algorithms.random_pred import NormalPredictor
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise.prediction_algorithms.matrix_factorization import SVD
from surprise.prediction_algorithms.knns import KNNWithMeans
from surprise.prediction_algorithms.slope_one import SlopeOne
import io
import surprise.Trainset

ModuleNotFoundError: ignored

In [ ]:
import surprise


In [ ]:
beers_key = pd.read_csv('https://raw.githubusercontent.com/kevin-o-lewis/beer-recommender/main/data/beers_key.csv', index_col=0)
prediction_table = pd.read_csv('https://raw.githubusercontent.com/kevin-o-lewis/beer-recommender/main/data/beer_ids_for_prediction.csv', index_col=0)
test = pd.read_csv('https://raw.githubusercontent.com/kevin-o-lewis/beer-recommender/main/data/test.csv', index_col=0)
train = pd.read_csv('https://raw.githubusercontent.com/kevin-o-lewis/beer-recommender/main/data/train.csv', index_col=0)

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
prediction_table

,user,beer_id,predicted_rating
0,None,47986,0
1,None,48213,0
2,None,48215,0
3,None,47969,0
4,None,64883,0
...,...,...,...
1518459,None,41749,0
1518461,None,38160,0
1518462,None,71234,0
1518463,None,43233,0


In [ ]:
reader = Reader(rating_scale=(1, 10))
suprise_train = Dataset.load_from_df(train, reader)
full_trainset = suprise_train.build_full_trainset()

algo=SVD()
algo.fit(full_trainset)

In [ ]:
prediction_table['user'] = 'changeup45'

#algo.predict()

for idx in prediction_table.index:
    prediction_table.loc[idx, ('predicted_rating')] = algo.predict('changeup45', prediction_table['beer_id'][idx])[3]

In [ ]:
top_5 = prediction_table.sort_values(by='predicted_rating', ascending=False)[0:5]['beer_id']
results = pd.DataFrame()
for beer_id in top_5:
    results = results.append(beers_key[beers_key['beer_beerid']==beer_id])
results

,beer_beerid,beer_name,brewery_name
748344,33127,Darkness,Surly Brewing Company
1510739,67760,Galaxy Imperial Single Hop IPA,Hill Farmstead Brewery
135413,47022,Hunahpu's Imperial Stout,Cigar City Brewing
1116266,2093,90 Minute IPA,Dogfish Head Brewery
310133,645,Trappistes Rochefort 10,Brasserie de Rochefort


In [ ]:
def load_data():
    import pandas as pd

    beers_key = pd.read_csv('https://raw.githubusercontent.com/kevin-o-lewis/beer-recommender/main/data/beers_key.csv', index_col=0)
    prediction_table = pd.read_csv('https://raw.githubusercontent.com/kevin-o-lewis/beer-recommender/main/data/beer_ids_for_prediction.csv', index_col=0)
    test = pd.read_csv('https://raw.githubusercontent.com/kevin-o-lewis/beer-recommender/main/data/test.csv', index_col=0)
    train = pd.read_csv('https://raw.githubusercontent.com/kevin-o-lewis/beer-recommender/main/data/train.csv', index_col=0)

    return beers_key,prediction_table,test,train

In [ ]:
def fit_to_trainset(algo):
    from surprise import Reader
    
    reader = Reader(rating_scale=(1, 10))
    suprise_train = Dataset.load_from_df(train, reader)
    full_trainset = suprise_train.build_full_trainset()
    algo = algo.fit(full_trainset)
    print('Fit complete.')
    return algo
    

In [ ]:
def recommend(algo, user):
    '''
    Parameters:
        pre-fit model
        username
    Returns:
        Top 5 recommended beers for that user
    '''
    
    prediction_table['user'] = user
    
    for idx in prediction_table.index:
        prediction_table.loc[idx, ('predicted_rating')] = algo.predict(user, prediction_table['beer_id'][idx])[3]

    top_5 = prediction_table.sort_values(by='predicted_rating', ascending=False)[0:5]['beer_id']
    results = pd.DataFrame()
    for beer_id in top_5:
        results = results.append(beers_key[beers_key['beer_beerid']==beer_id])

    return results
    
    # fit algo on train set (surprise trainset object)
    # add user to beer_ids_for_prediction
    # run algo.predict on beer_ids_for_prediction
    # sort results, return top 5 beerids
    # translate beer ids to beer names
    # return beer names
    

In [ ]:
def score(algo, testset):
    #pass in a pre-fit model
    #make a full ranked prediction list for each user
    #look up each prediction in order in the test set
    #join the prediction and actual
    #compare the order of the ranked beers
    top_scores = []
    for user in testset['review_profilename'].unique():
        prediction_table['user'] = user
        for idx in prediction_table.index:
            prediction_table.loc[idx, ('predicted_rating')] = algo.predict(user, prediction_table['beer_id'][idx])[3]

        ranked_recommendations = prediction_table.sort_values(by='predicted_rating', ascending=False)
        user_test_scores = test[test['review_profilename']==user]
        num_scores = 1
        for beer_id in ranked_recommendations['beer_id'].values:
            if beer_id in list(user_test_scores['beer_beerid']):
                num_scores -= 1
                top_scores.append(int(user_test_scores.review_overall[user_test_scores['beer_beerid']==beer_id].values[0]))
                if num_scores == 0:
                    break
            else: continue

    return print('Average rating of top recommendations:', sum(top_scores)/len(top_scores))

In [ ]:
beers_key,prediction_table,test,train = load_data()

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
model = fit_to_trainset(NormalPredictor())

Fit complete.


In [54]:
predictions = model.test(build_testset(test))

NameError: ignored

In [ ]:
score(model, test)

KeyboardInterrupt: ignored